# 5 Flower classification Using Deep Learning and Machine Learning Model

In [1]:
import tensorflow as tf
import numpy as num
import os
import glob
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import StratifiedShuffleSplit
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier

# Load Train , Test and Validation data

In [2]:
import pickle
read_train=open("training_5flowerdata.pickle","rb")
load_x_train=pickle.load(read_train)
read_train_label=open("training_5flower_label.pickle","rb")
load_y_train=pickle.load(read_train_label)

In [3]:
read_val=open("Dev_5flowerdata.pickle","rb")
load_x_val=pickle.load(read_val)
read_val_label=open("Dev_5flower_label.pickle","rb")
load_y_val=pickle.load(read_val_label)

In [4]:
read_test=open("test_5flowerdata.pickle","rb")
load_x_test=pickle.load(read_test)
read_test_label=open("test_5flowerlabel.pickle","rb")
load_y_test=pickle.load(read_test_label)

In [5]:
print("training shape:",load_x_train.shape)
print("test shape:",load_x_test.shape)
print("validation data shape:",load_x_val.shape)

training shape: (6475, 224, 224, 3)
test shape: (289, 224, 224, 3)
validation data shape: (894, 224, 224, 3)


# MobileNet Model

In [6]:
mobile=tf.keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)

In [7]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [8]:
custoom_layer=mobile.layers[-7].output

In [9]:
added_layer=Flatten()(custoom_layer)

In [10]:
output_layer=Dense(5,activation="softmax")(added_layer)

In [11]:
mobi_net_model=Model(inputs=mobile.input,outputs=output_layer)

In [12]:
mobi_net_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

# Freezeing few layers

In [13]:
for layers in mobi_net_model.layers[:-23]:
    layers.trainable=False

In [14]:
mobi_net_model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [17]:
with tf.device('/cpu:0'):
    hist=mobi_net_model.fit(load_x_train,load_y_train,epochs=5,validation_split=0.05,batch_size=25)

Epoch 1/5
247/247 [==============================] - 213s 853ms/step - loss: 20.4994 - accuracy: 0.9172 - val_loss: 42.0778 - val_accuracy: 0.8765
Epoch 2/5
247/247 [==============================] - 213s 862ms/step - loss: 10.0776 - accuracy: 0.9537 - val_loss: 36.0568 - val_accuracy: 0.9228
Epoch 3/5
247/247 [==============================] - 210s 851ms/step - loss: 8.9332 - accuracy: 0.9649 - val_loss: 1.5918 - val_accuracy: 0.9846
Epoch 4/5
247/247 [==============================] - 209s 847ms/step - loss: 5.5596 - accuracy: 0.9745 - val_loss: 24.5885 - val_accuracy: 0.9537
Epoch 5/5
247/247 [==============================] - 209s 847ms/step - loss: 4.4296 - accuracy: 0.9805 - val_loss: 13.4582 - val_accuracy: 0.9599


# Unfreezing the freezed layers

In [18]:
for layers in mobi_net_model.layers[:-23]:
    layers.trainable=True

In [19]:
mobi_net_model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=["accuracy"])

In [20]:
with tf.device('/cpu:0'):
    mobi_net_model.fit(load_x_train,load_y_train,epochs=5,validation_split=0.05,batch_size=25)

Epoch 1/5
247/247 [==============================] - 565s 2s/step - loss: 26.4559 - accuracy: 0.9059 - val_loss: 1222.2245 - val_accuracy: 0.4691
Epoch 2/5
247/247 [==============================] - 563s 2s/step - loss: 15.0884 - accuracy: 0.9262 - val_loss: 177.6806 - val_accuracy: 0.7346
Epoch 3/5
247/247 [==============================] - 575s 2s/step - loss: 10.4378 - accuracy: 0.9385 - val_loss: 195.6433 - val_accuracy: 0.6481
Epoch 4/5
247/247 [==============================] - 564s 2s/step - loss: 7.8020 - accuracy: 0.9501 - val_loss: 434.2299 - val_accuracy: 0.6636
Epoch 5/5
247/247 [==============================] - 557s 2s/step - loss: 7.0295 - accuracy: 0.9568 - val_loss: 288.7785 - val_accuracy: 0.6080


# RANFOM FOREST

In [21]:
RF_model=RandomForestClassifier(n_estimators=50,random_state=42)

In [22]:
model_feature=Model(inputs=mobile.input,outputs=added_layer)

In [23]:
model_feature.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [24]:
with tf.device('/cpu:0'):
    extracted=model_feature.predict(load_x_train)

In [25]:
RF_model.fit(extracted,load_y_train)

RandomForestClassifier(n_estimators=50, random_state=42)

In [26]:
val_pred=RF_model.predict(extracted)

# Accuracy

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(val_pred,load_y_train)

0.9998455598455599

In [28]:
from sklearn.model_selection import cross_val_score
cross_val_score(RF_model,extracted,load_y_train,cv=5,scoring="accuracy")

array([0.96988417, 0.96679537, 0.95830116, 0.95907336, 0.96293436])

#  Grid search Hyperparameter tuning

In [29]:
from sklearn.model_selection import GridSearchCV
pram_grid=[
    {'n_estimators':[70,80,90],"bootstrap":[False,True]}
]
grid=GridSearchCV(RF_model,pram_grid,cv=5,scoring="accuracy",return_train_score=True)

In [30]:
grid.fit(extracted,load_y_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=50, random_state=42),
             param_grid=[{'bootstrap': [False, True],
                          'n_estimators': [70, 80, 90]}],
             return_train_score=True, scoring='accuracy')

In [31]:
grid.best_params_

{'bootstrap': False, 'n_estimators': 70}

In [32]:
grid.best_score_

0.968803088803089

In [33]:
grid.best_estimator_

RandomForestClassifier(bootstrap=False, n_estimators=70, random_state=42)